## We should watch [this YT video](https://www.youtube.com/watch?v=VkjqViooMtQ) to know how to set up the IAM and connector part to Azure.

## Read daily and monthly data files from Azure [Data Lake Storage] and build corresponding bronze tables for them.

In [0]:
from functools import reduce

# configuration variables
years = list(range(2010, 2025))
folders = [("daily", "D"), ("monthly", "M")]

# for path to Azure Data Lake Storage Gen2
for folder in folders:
    file_paths = [
        f"abfss://twstocks@kenspractice.dfs.core.windows.net/{folder[0]}/2330_{folder[1]}_{year}.parquet"
        for year in years
    ]

    # read as a list of DataFrame 
    dfs = [spark.read.parquet(path) for path in file_paths]

    # merge DataFrame using reduce and unionByName
    df_combined = reduce(lambda df1, df2: df1.unionByName(df2), dfs)
    df_combined = df_combined.orderBy("date")

    # df_combined.printSchema()
    
    # display(df_combined)

    df_combined.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable(f"kenworkspace.tw_stocks_db.bronze_{folder[0]}")

## Query the table [bronze_daily]

In [0]:
%sql
SELECT *
FROM kenworkspace.tw_stocks_db.bronze_daily

## Query the table [bronze_monthly]

In [0]:
%sql
SELECT *
FROM kenworkspace.tw_stocks_db.bronze_monthly